# Basic transfer learning with cats and dogs data



### 1. Importing the dataset using `urllib()` class. [Run Once]

In [1]:
import urllib.request
url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip"
filename = 'cats-vs-dogs.zip'
# I have already downloaded it, so it is commented out.
# urllib.request.urlretrieve(url, filename=filename)

### 2. Extract the images with `zipfile()` class 

n.b- use your suitable base directory.

In [2]:
import zipfile
zipped = zipfile.ZipFile(filename, mode='r')
base_dir = r'C:\Nerd Stuff\1.DS\#uploadable_codes\CatVsDog'
zipped.extractall(base_dir)
zipped.close()

### 3(a). Checking the images number

In [3]:
import os

print(len(os.listdir(base_dir+r'\PetImages\Cat')))
print(len(os.listdir(base_dir+r'\PetImages\Dog')))

12501
12501


### 3(b) Making Directories

In [4]:

try: 
    os.mkdir(base_dir+r'\cats-v-dogs')
    os.mkdir(base_dir+r'\cats-v-dogs\training')
    os.mkdir(base_dir+r'\cats-v-dogs\training\cats')
    os.mkdir(base_dir+r'\cats-v-dogs\training\dogs')
    os.mkdir(base_dir+r'\cats-v-dogs\testing')
    os.mkdir(base_dir+r'\cats-v-dogs\testing\cats')
    os.mkdir(base_dir+r'\cats-v-dogs\testing\dogs')
except OSError:
    pass

### 4. Splititng the train and test data.
- The following code put first checks if an image file is empty (zero length)
- Of the files that are not empty, it puts 90% of the data into the training set, and 10% into the test set.

In [5]:
import random
from shutil import copyfile

def split_data(source_dir, train_dir, test_dir, split_size):
    ## eliminating zero-sized images and storing their filenames
    files = []
    for filename in os.listdir(source_dir):
        file = source_dir+filename
        if os.path.getsize(file)>0:
            files.append(filename)
        else:
            print(f'{filename} is zero sized, so ignoring')
        
    trainset_len = int(len(files)*split_size)
    testset_len = int(len(files)-trainset_len)
    shuffled_file = random.sample(files, len(files))
    train_set = shuffled_file[:trainset_len]
    test_set = shuffled_file[trainset_len:]

    print(trainset_len, testset_len)
    #copying train_set to trainig_dir
    for filename in train_set:
        curr_file = source_dir+filename
        target_file = train_dir+filename
        copyfile(curr_file, target_file)
    # copying test_set to test_dir
    for filename in test_set:
        curr_file = source_dir+filename
        target_file = test_dir+filename
        copyfile(curr_file, target_file)

In [12]:
cat_source_dir = base_dir + r'\PetImages\Cat\\'
dog_source_dir = base_dir + r'\PetImages\Dog\\'

cat_train_dir = base_dir + r'\cats-v-dogs\training\cats\\'
dog_train_dir = base_dir + r'\cats-v-dogs\training\dogs\\'

cat_test_dir = base_dir + r'\cats-v-dogs\testing\cats\\'
dog_test_dir = base_dir + r'\cats-v-dogs\testing\dogs\\'

split_size=0.9
split_data(cat_source_dir, cat_train_dir, cat_test_dir, split_size)
split_data(dog_source_dir, dog_train_dir, dog_test_dir, split_size)


666.jpg is zero sized, so ignoring
11250 1250
11702.jpg is zero sized, so ignoring
11250 1250


In [13]:
print(f'Training cat images : {len(os.listdir(cat_train_dir))}')
print(f'Training dog images : {len(os.listdir(dog_train_dir))}')
print(f'Testing cat images : {len(os.listdir(cat_test_dir))}')
print(f'Testing dog images : {len(os.listdir(dog_test_dir))}')

Training cat images : 11250
Training dog images : 11250
Testing cat images : 1250
Testing dog images : 1250


### 5. Data(Image) Augmentation 
Here, we'll use the `ImageDataGenerator` to perform data augmentation.  
- First create an instance of the `ImageDataGenerator` class.
- Using the ImageDataGenerator object, call the method `flow_from_directory` and set the hyper-params

- Things like rotating and flipping the existing images allows you to generate training data that is more varied, and can help the model generalize better during training.  
- We can also use the data generator to apply data augmentation to the validation set.

In [21]:
train_dir = base_dir + r'\cats-v-dogs\training\\'
validation_dir = base_dir + r'\cats-v-dogs\testing\\'
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, 
shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
train_gen = train_datagen.flow_from_directory(train_dir, batch_size=100, class_mode='binary', target_size=(150,150))

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_gen = validation_datagen.flow_from_directory(validation_dir, batch_size=100, class_mode='binary', target_size=(150, 150))

Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


### 6. Get the `InceptionV3` model for tranfer learning.

#### a. download pretrained weights

In [22]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
# urllib.request.urlretrieve(weights_url, weights_file)

('inception_v3.h5', <http.client.HTTPMessage at 0x17ba359a040>)

#### b. Instantiate the pre_trained_model 

In [24]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape=(150,150,3), include_top=False, weights=None)

#### c. load the pretrained weights

In [25]:
pre_trained_model.load_weights(weights_file)

#### d. Freeze the layers

In [26]:
for layer in pre_trained_model.layers:
    layer.trainable = False

#### e. Get the last (suitable) layer.

In [27]:
pre_trained_model.summary()

67[0][0]              
__________________________________________________________________________________________________
conv2d_69 (Conv2D)              (None, 7, 7, 192)    147456      average_pooling2d_6[0][0]        
__________________________________________________________________________________________________
batch_normalization_60 (BatchNo (None, 7, 7, 192)    576         conv2d_60[0][0]                  
__________________________________________________________________________________________________
batch_normalization_63 (BatchNo (None, 7, 7, 192)    576         conv2d_63[0][0]                  
__________________________________________________________________________________________________
batch_normalization_68 (BatchNo (None, 7, 7, 192)    576         conv2d_68[0][0]                  
__________________________________________________________________________________________________
batch_normalization_69 (BatchNo (None, 7, 7, 192)    576         conv2d_69[0][0]      

In [34]:
last_layer = pre_trained_model.get_layer('mixed7')
print(f'Shape of last layer:{last_layer.output_shape}')
last_output = last_layer.output

Shape of last layer:(None, 7, 7, 768)


### 7. Adding Layers

In [38]:
import tensorflow as tf
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=pre_trained_model.input, outputs=x)

### 8. Training the model.

In [40]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.001), metrics=['acc'], loss='binary_crossentropy')
history = model.fit(train_gen, validation_data=validation_gen, epochs=2, verbose=1)

Epoch 1/2
 66/225 [=======>......................] - ETA: 10:28 - loss: 2.0629 - acc: 0.8219C:\Users\dewan\AppData\Local\Programs\Python\Python38\lib\site-packages\PIL\TiffImagePlugin.py:793: UserWarning: Truncated File Read
  warnings.warn(str(msg))
225/225 [==============================] - 952s 4s/step - loss: 1.0667 - acc: 0.8640 - val_loss: 0.1888 - val_acc: 0.9380
Epoch 2/2
225/225 [==============================] - 857s 4s/step - loss: 0.1853 - acc: 0.9279 - val_loss: 0.0890 - val_acc: 0.9664
